# Silver Layer Data Transformation

### Netflix Cast data

In [1]:
print("Silver Layer Transformation Starting...")

Silver Layer Transformation Starting...


In [ ]:
cast_df = spark.read.format("delta")\
            .option("header", "true")\
            .option("inferSchema", "true")\
            .load(f"abfss://silver@datalakeprojectnetflix.dfs.core.windows.net/netflix_cast")

### Netflix Category data

In [ ]:
category_df = spark.read.format("delta")\
            .option("header", "true")\
            .option("inferSchema", "true")\
            .load(f"abfss://silver@datalakeprojectnetflix.dfs.core.windows.net/netflix_category")

### Netflix Countries data

In [ ]:
countries_df = spark.read.format("delta")\
            .option("header", "true")\
            .option("inferSchema", "true")\
            .load(f"abfss://silver@datalakeprojectnetflix.dfs.core.windows.net/netflix_countries")

### Netflix Directors data

In [ ]:
directors_df = spark.read.format("delta")\
            .option("header", "true")\
            .option("inferSchema", "true")\
            .load(f"abfss://silver@datalakeprojectnetflix.dfs.core.windows.net/netflix_directors")

### Netflix Titles data

In [ ]:
titles_df = spark.read.format("csv")\
            .option("header", "true")\
            .option("inferSchema", "true")\
            .load(f"abfss://raw@datalakeprojectnetflix.dfs.core.windows.net")

In [ ]:
titles_df.printSchema()
display(titles_df)

## Transformation

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
titles_df = titles_df.fillna({"duration_minutes": 0, "duration_seasons": 0})
display(titles_df)

In [ ]:
titles_df = titles_df.withColumn("duration_minutes", col("duration_minutes").cast(IntegerType()))\
                     .withColumn("duration_seasons", col("duration_seasons").cast(IntegerType()))
display(titles_df)


In [ ]:
titles_df = titles_df.withColumn("ShortTitle", split(col("title"), ":").getItem(0))
display(titles_df)

In [ ]:
titles_df = titles_df.withColumn("rating_new", split(col("rating"), "-").getItem(0))
display(titles_df)

In [ ]:
from pyspark.sql.window import Window

In [ ]:
window = Window.orderBy(col("duration_minutes").desc())
titles_df = titles_df.withColumn("rank_duration_minutes", dense_rank().over(window))
display(titles_df)

In [ ]:
titles_df = titles_df.dropDuplicates(["show_id"])
display(titles_df)

In [ ]:
titles_df = titles_df.fillna({"rating_new":"R"})

In [ ]:
titles_df.write.format("delta").mode("overwrite")\
    .option("path", f"abfss://silver@datalakeprojectnetflix.dfs.core.windows.net/netflix_titles")\
    .save()

In [2]:
print("Silver Layer Transformation completed.")

Silver Layer Transformation completed.
